# COBRApy tutorial

This tutorial is intended to cover the basics of cobrapy usage. It is derived from cobra documentation, available __[here](https://cobrapy.readthedocs.io/en/latest/index.html)__


# 1) Getting Started:

### 1.1) Load and inspect a model

To demonstrate this first example we will use a small test model, which is already included in the COBRA packeage inside the `cobra.test` module: 

The function to read and load a SBML model is instead located in the `cobra.io` module  

In [1]:
import cobra
import cobra.test  # <- explicitly import cobra.test or it will not work
from os.path import join

data_dir = cobra.test.data_dir
model = cobra.io.read_sbml_model(join(data_dir, "mini_fbc2.xml"))


Try to run `read_sbml_model()` function, changing the argument to match the path where you saved your model ( Recon2 or SKM )  

The model has several "attributes", that are accessed through the dot notation:

In [ ]:
model.  # <-- press <TAB> key to inspect model attributes

`model.reactions`, `model.metabolites` and `model.genes` are attributes that contain lists of `cobra.Reaction`, `cobra.Metabolite` and `cobra.Gene` objects respectively:

In [2]:
print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

18
23
31


We can inspect the content of these list with a for loop: 

In [3]:
for m in model.metabolites: print(m)

13dpg_c
2pg_c
3pg_c
adp_c
atp_c
dhap_c
f6p_c
fdp_c
g3p_c
g6p_c
glc__D_e
h2o_c
h2o_e
h_c
h_e
lac__D_c
lac__D_e
nad_c
nadh_c
pep_c
pi_c
pi_e
pyr_c


In [4]:
model.compartments


{'c': 'cytosol', 'e': 'extracellular'}

This small example model only has two compartments: cytosol and extracellular. 

Models describing more complex organisms will have more compartments:

In [5]:
ecoli_model = cobra.test.create_test_model("ecoli")
ecoli_model.compartments

{'c': 'Cytoplasm', 'e': 'Extracellular', 'p': 'Periplasm'}

### 1.2) Metabolites

Reaction and Metabolites objects can be retrieved by their 'ID' using the .get_by_id() function:

In [8]:
nadh = model.metabolites.get_by_id('nadh_c')

Metabolite object stores information like the metabolite's full name , compartment, charge and formula:


In [11]:
print(nadh.name)
print(nadh.compartment)
print(nadh.charge)
print(nadh.formula)

Nicotinamide adenine dinucleotide - reduced
c
-2
C21H27N7O14P2


or we can view in which reactions nadh is involved:

In [13]:
nadh.reactions

frozenset({<Reaction GAPD at 0x7f78bccd7990>,
           <Reaction LDH_D at 0x7f78bccd7a90>})

### 1.3) Reactions

Working with reactions objects is similar:
We can access them by ID:

In [15]:
PYK = model.reactions.get_by_id('PYK')

The object stores informations like  reaction's full name , reaction catalyzed and reversibility:

In [16]:
print(PYK.name)
print(PYK.reaction)
print(PYK.reversibility)

pyruvate kinase
adp_c + h_c + pep_c --> atp_c + pyr_c
False


We can also view ( and change ) reactions' bounds:

In [19]:
print(PYK.bounds)
PYK.bounds = -500 , 500

(-500, 500)


In [ ]:
PYK.bounds = -500 , 500
print(PYK.bounds)

# 2) Building and expanding a model:

This example demonstrates how to create metabolites and reaction objects, and how to add them to a empty test model.
First, create the empty test model:

In [2]:
from cobra import Model,Metabolite,Reaction
test_model = Model("test model")

Create a Reaction object (Pyruvate Dehidrogenase) and define its attributes:

In [ ]:
PDH = Reaction(id = "PDH", 
               name = "Pyruvate dehidrogenase", 
               subsystem = "central carbon metabolism",
               lower_bound = 0, 
               upper_bound = 1000)

We need to create metabolites objects as well. 
If we were using an existing model, we could use `model.metabolites.get_by_id()` to get the appropriate Metabolite objects instead.

In [ ]:
accoa = Metabolite(id = 'accoa_c',
                   name = 'Acetyl-CoA',
                   compartment = 'c')

co2 = Metabolite(id = 'co2_c',
                   name = 'CO2',
                   compartment = 'c')

coa = Metabolite(id = 'coa_c',
                   name = 'Co-enzyme A',
                   compartment = 'c')

nad = Metabolite(id = 'nad_c',
                   name = 'Nicotinamide adenine dinucleotide',
                   compartment = 'c')

nadh = Metabolite(id = 'nadh_c',
                   name = 'Nicotinamide adenine dinucleotide - reduced',
                   compartment = 'c')

pyr = Metabolite(id = 'pyr_c',
                   name = 'pyruvate',
                   compartment = 'c')

Add the metabolites and their coefficients to the Reaction object: 
Attention! This function needs a dict as argument:  { x : y , ... } 

In [ ]:
PDH.add_metabolites({ 
        coa   : -1,
        nad   : -1,
        pyr   : -1,
        accoa :  1,
        co2   :  1,
        nadh  :  1,
        })

At this point the model is still empty:

In [ ]:
print('%i reactions initially' % len(test_model.reactions))
print('%i metabolites initially' % len(test_model.metabolites))

When we add the reaction to the model, it will also add all associated metabolites:

In [ ]:
test_model.add_reaction(PDH)

print('%i reaction' % len(test_model.reactions))
print('%i metabolites' % len(test_model.metabolites))

# 3) Flux Balance Analysis

In this example we will use a e.coli model: 

In [26]:
import cobra.test
model = cobra.test.create_test_model("ecoli")

We set the objective function:

In [27]:
model.objective = 'Ec_biomass_iJO1366_WT_53p95M'

... and set the LP solver:

if you installed gurobi on your PC as indicated in the installation tutorial, you can set it in this way: 

In [28]:
model.solver = 'glpk' # or 'gurobi'

FBA optimization problem can be solved calling `model.optimize()`. 
This will maximize or minimize (maximizing is the default) flux through the objective reaction.

In [37]:
solution = model.optimize()

`model.optimize()` will return a Solution object. A solution object has several attributes:

In [30]:
#the objective value
print('Obj.value = ', solution.objective_value)

print('------------------------')
#the flux distribution
print(solution.fluxes.head(10)) # <---- head(10) prints only first 10 fluxes

print('------------------------')
# solver status (e.g. 'optimal' or 'unfeasible')
print('solution is :',  solution.status)

('Obj.value = ', 0.9865144469529751)
------------------------
DM_4CRSOL                         0.000220
DM_5DRIB                          0.000228
DM_AACALD                         0.000000
DM_AMOB                           0.000002
DM_MTHTHF                         0.001322
DM_OXAM                           0.000000
Ec_biomass_iJO1366_WT_53p95M      0.986514
Ec_biomass_iJO1366_core_53p95M    0.000000
EX_12ppd__R_e                     0.000000
EX_12ppd__S_e                     0.000000
Name: fluxes, dtype: float64
------------------------
('solution is :', 'optimal')


# 4) Inspecting FBA solution

The summary() method displays information on the input and output behavior of the model, along with the optimized objective:

In [32]:
model.summary()  #<--- must be called AFTER model.optimize() 

IN FLUXES             OUT FLUXES           OBJECTIVES
--------------------  -------------------  ----------------------
o2_e       17.6       h2o_e     45.1       Ec_biomass_i...  0.987
nh4_e      10.4       co2_e     19.7
glc__D_e   10         h_e        8.75
pi_e        0.915     5mtr_e     0.00665
so4_e       0.249     mththf_c   0.00132
k_e         0.183     5drib_c    0.000228
fe2_e       0.0153    4crsol_c   0.00022
mg2_e       0.00814   amob_c     2e-06
ca2_e       0.00488   meoh_e     2e-06
cl_e        0.00488
cu2_e       0.000665
mn2_e       0.000649
zn2_e       0.00032
ni2_e       0.000303
cbl1_e      0.00022
mobd_e      0.000135
cobalt2_e   2.4e-05


Input-output behavior of individual metabolites can also be inspected. 

E.g. Checking (cytosolic) NADH behavior can be useful to inspect the redox balance of the cell:

In [33]:
model.metabolites.nadh_c.summary()

PRODUCING REACTIONS -- Nicotinamide adenine dinucleotide - reduced (nadh_c)
---------------------------------------------------------------------------
%       FLUX  RXN ID    REACTION
---  -------  --------  --------------------------------------------------
45%  15.9     GAPD      g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
22%   7.85    PDH       coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
14%   4.93    MDH       mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
11%   3.93    AKGDH     akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
5%    1.62    PGCD      3pg_c + nad_c --> 3php_c + h_c + nadh_c
1%    0.432   IPMD      3c2hmp_c + nad_c --> 3c4mop_c + h_c + nadh_c
1%    0.233   IMPD      h2o_c + imp_c + nad_c --> h_c + nadh_c + xmp_c
1%    0.182   HISTD     h2o_c + histd_c + 2.0 nad_c --> 3.0 h_c + his__...
0%    0.132   PPND      nad_c + pphn_c --> 34hpp_c + co2_c + nadh_c
0%    0.0554  GLYCL     gly_c + nad_c + thf_c --> co2_c + mlthf_c + nad...

CONSUMING REACTIONS -- Nicoti

Checking instead cytosolic atp behavior will give us a sense of the cell's energy production and consumption:

In [34]:
model.metabolites.atp_c.summary()


PRODUCING REACTIONS -- ATP (atp_c)
----------------------------------
%       FLUX  RXN ID      REACTION
---  -------  ----------  --------------------------------------------------
70%  56       ATPS4rpp    adp_c + 4.0 h_p + pi_c <=> atp_c + h2o_c + 3.0 h_c
20%  15.9     PGK         3pg_c + atp_c <=> 13dpg_c + adp_c
5%    4.13    PPKr        atp_c + pi_c <=> adp_c + ppi_c
4%    3.42    SUCOAS      atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c

CONSUMING REACTIONS -- ATP (atp_c)
----------------------------------
%       FLUX  RXN ID      REACTION
---  -------  ----------  --------------------------------------------------
67%  53.4     Ec_biom...  0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-0...
9%    6.99    PFK         atp_c + f6p_c --> adp_c + fdp_c + h_c
4%    3.15    ATPM        atp_c + h2o_c --> adp_c + h_c + pi_c
4%    3.09    ADK1        amp_c + atp_c <=> 2.0 adp_c
2%    1.71    GLNS        atp_c + glu__L_c + nh4_c --> adp_c + gln__L_c +...
1%    1.03    ASPK        asp

# 5) Save and export a model:


Model can be saved and exported through the `write_sbml_model()` function, found in the `cobra.io` sub-module. 

In [36]:
cobra.io.write_sbml_model(model, "test_model_AC.xml")

# 6) Visualizing results : graph view

Visualizing FBA results

This function creates a file that can be imported with networkX and visualized as a graph

In [ ]:
def save_flow_graph(model, solution, filename):
    
    nnz_sol = solution.fluxes.iloc[solution.fluxes.nonzero()]
    RXN_Nodes = [r for r , v in nnz_sol.iteritems()]
    met_edges_in = { x : [m.id for m in model.reactions.get_by_id(x).reactants] for x in RXN_Nodes}
    met_edges_out = { x : [m.id for m in model.reactions.get_by_id(x).products] for x in RXN_Nodes}
    
    nodef=open(filename, 'w')
    for rID , v in met_edges_in.items():
        for met in v: 
            nodef.write(rID+'\t'+met)
            nodef.write("\n")
    for rID , v in met_edges_out.items():
        for met in v: 
            nodef.write(rID+'\t'+met)
            nodef.write("\n")
    nodef.close()
    

# 6) Visualizing results : ReconMap

Visualizing FBA results
This function creates a layout file that can be visualized on ReconMap

In [ ]:
import pandas as pd

def viz(solution, filename):
    nnz_sol = solution.fluxes.iloc[solution.fluxes.nonzero()]
    df = pd.DataFrame(columns = ['reactionIdentifier','lineWidth','color'])
    df['reactionIdentifier'] = nnz_sol.index
    df['lineWidth'] = (nnz_sol - (min(nnz_sol)) / (max(nnz_sol) - min(nnz_sol)) <- #Normalized fluxes 
    df['color'] = '#FF0000'# red 
    df.to_csv(filename, sep= '\t', header = True, index = False)